In [83]:
from os import listdir
from os.path import isfile, join

import csv
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import requests
import re

In [84]:
SPOTIFY_CLIENT_ID ="8960c84e74964817bfb927e83f6e2b59"
SPOTIFY_SECRET ="3917b5e7cc3342189238c7290f99be3c"
REDIRECT_URL = "http://localhost:8080/"

In [85]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-public",
        redirect_uri=REDIRECT_URL,
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_SECRET,
        cache_path="token.txt"
    )
)

In [86]:
user_id = "31bezwu6mascvq773ob6sb7bikjy"

In [87]:
'''
spotify_song_uris = [
                    "spotify:track:5WZDIh4WVkcHItK7y9E8Fl",
                    "spotify:track:2rf9i0Enr8cw1JRME8Rsvq"
                    ]
'''

spotify_song_uris = [
                    "5WZDIh4WVkcHItK7y9E8Fl",
                    "2rf9i0Enr8cw1JRME8Rsvq"
                    ]

In [88]:
bookName = 'bookTest'

playlistName = bookName + ' playlist'

In [89]:
playlists = []

os = 0
stop = False

while True:
    segment = sp.user_playlists(user_id, limit = 50, offset = os)
    
    numItems = len(segment['items'])
    
    for item in segment['items']:
        if item['name'] == playlistName:
            stop = True
            break
    
    if stop:
        break
    
    playlists = playlists + segment['items']
    
    if numItems < 50 or segment['items'] == []:
        break
    
    os += 50

In [90]:
if not stop:
    desc = "Generated playlist for " + bookName
    
    playlist = sp.user_playlist_create(user=f"{user_id}", name= playlistName, public=True,
                                          description=desc)

    playlistId = playlist["id"]

    link = playlist['external_urls']['spotify']

    sp.user_playlist_add_tracks(user=f"{user_id}", playlist_id=playlistId, tracks=spotify_song_uris)
    
    print(link)
else:
    print('playlist_exists_already')

playlist_exists_already
